<a href="https://colab.research.google.com/github/Janek9527/KI_Lab_Project/blob/master/LAB_Project_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!rm -r fishygame
!rm fishygame.zip
!wget https://kilab.blob.core.windows.net/code/fishygame.zip
!unzip fishygame.zip

--2021-06-18 07:15:30--  https://kilab.blob.core.windows.net/code/fishygame.zip
Resolving kilab.blob.core.windows.net (kilab.blob.core.windows.net)... 20.38.118.132
Connecting to kilab.blob.core.windows.net (kilab.blob.core.windows.net)|20.38.118.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 509939 (498K) [application/x-zip-compressed]
Saving to: ‘fishygame.zip’

fishygame.zip       100%[===================>] 497.99K  1.12MB/s    in 0.4s    

2021-06-18 07:15:31 (1.12 MB/s) - ‘fishygame.zip’ saved [509939/509939]

Archive:  fishygame.zip
   creating: fishygame/
  inflating: fishygame/.gitignore    
   creating: fishygame/.vscode/
 extracting: fishygame/.vscode/settings.json  
  inflating: fishygame/agent.py      
   creating: fishygame/constants/
  inflating: fishygame/constants/constants.py  
 extracting: fishygame/constants/__init__.py  
   creating: fishygame/constants/__pycache__/
  inflating: fishygame/constants/__pycache__/constants.cpython-38.pyc 

In [15]:
import torch
import torch.nn as nn
from torch import optim
import random
import numpy
import math
from collections import namedtuple, deque
from itertools import count
import torch.nn.functional as F
!pip3 install arcade

In [16]:
%%bash

# install required system dependencies
apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
pip install pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists...
Building dependency tree...
Reading state information...
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [17]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(960, 540))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '960x540x24', ':1003'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '960x540x24', ':1003'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [18]:
import sys
sys.path.append("fishygame")

In [19]:
from fishygame import game_class
from game_class import Game

In [20]:
device = torch.device("cpu")

In [21]:
class Net(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.flatten1 = torch.nn.Flatten()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, H)
        self.linear3 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x = self.flatten1(x)
        h_relu1 = F.relu(self.linear1(x))
        h_relu2 = F.relu(self.linear2(h_relu1))
        return self.linear3(h_relu2)


Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [22]:
state_dim = 15 * 5
hidden_dim = 100
action_count = 4

policy_net = Net(state_dim, hidden_dim, action_count)
target_net = Net(state_dim, hidden_dim, action_count)

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)
activation = torch.nn.Softmax(dim=1)

In [23]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)

    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [24]:
steps_done = 0
n_actions = action_count
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            #print(policy_net(state))
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

episode_durations = []

In [ ]:
initialState = [[0.0e+00, 0.0e+00, 4.8e+02, 2.7e+02, 3.0e-01],        
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00]]

num_episodes = 10000
n_actions = action_count
for i_episode in range(num_episodes):
    # Initialize the environment and state
    if (i_episode % 100) == 0:
        torch.save(target_net, f"./modeldist{i_episode}")
    game = Game()
    state = torch.tensor([initialState]).float()
    localmemory = deque()
    for t in count():
        # Select and perform an action
        #print(f"Episode: {i_episode}, t: {t}")
        #print(state)
        action = select_action(state)
        state_next, reward, done = game.on_update(action.item())
        reward = torch.tensor([reward])

        # Observe new state
        if not done:
            next_state = torch.tensor([state_next]).float()
        else:
            next_state = None

        memory.push(state, action, next_state, reward)

        if reward == 100 or reward == -1000:
          print(reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            #plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([100])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([100])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-1000])
tensor([-100

In [ ]:
torch.save(policy_net, "./modeldistnew10000")